# HW3: Netflix Data Analysis

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import scipy.sparse
import random

In [2]:
ratings = scipy.sparse.load_npz('sample_matrix.npz')

In [3]:
test_value = np.load('test_value.npz')['arr_0']
test_movie = np.load('test_movie.npz')['arr_0']
test_user = np.load('test_user.npz')['arr_0']
train_value = np.load('train_value.npz')['arr_0']
train_movie = np.load('train_movie.npz')['arr_0']
train_user = np.load('train_user.npz')['arr_0']

In [23]:
# Get features
movie_avg = np.zeros(5000)
movie_revs = np.zeros(5000)
user_avg = np.zeros (10000)
user_revs = np.zeros(10000)
for i in range(5000):
    movie_slice = ratings[i, :]
    movie_avg[i] = csr_matrix.mean(movie_slice)
    movie_revs[i] = csr_matrix.count_nonzero(movie_slice)
for i in range(10000):
    user_slice = ratings[:, i]
    user_avg[i] = csr_matrix.mean(user_slice)
    user_revs[i] = csr_matrix.count_nonzero(user_slice)

In [25]:
#Movie avg, user avg, # movie reviews, # user reviews, # positive movie reviews, # negative movie review
# #positive user reiews, # negative user reviews
movie_avg_train = np.zeros(len(train_value))
user_avg_train = np.zeros(len(train_value))
movie_revs_train = np.zeros(len(train_value))
user_revs_train = np.zeros(len(train_value))

for i in range(len(train_value)):
    movie_avg_train[i] = movie_avg[train_movie[i]]
    user_avg_train[i] = user_avg[train_user[i]]
    movie_revs_train[i] = movie_revs[train_movie[i]]
    user_revs_train[i] = user_revs[train_user[i]]

features_train = np.vstack((movie_avg_train, user_avg_train, movie_revs_train, user_revs_train)).T

movie_avg_test = np.zeros(len(test_value))
user_avg_test = np.zeros(len(test_value))
movie_revs_test = np.zeros(len(test_value))
user_revs_test = np.zeros(len(test_value))

for i in range(len(test_value)):
    movie_avg_test[i] = movie_avg[test_movie[i]]
    user_avg_test[i] = user_avg[test_user[i]]
    movie_revs_test[i] = movie_revs[test_movie[i]]
    user_revs_test[i] = user_revs[test_user[i]]

features_test = np.vstack((movie_avg_test, user_avg_test, movie_revs_test, user_revs_test)).T

In [16]:
def MSE(predictions, test_set):
    summation = 0
    for i in range(len(predictions)):
        summation += (predictions[i] - test_set[i])**2
    return summation/len(predictions)

In [17]:
def MSE_non_zero(predictions, test_set):
    summation = 0
    non_zero = 0
    for i in range(len(predictions)):
        value = test_set[i]
        if value > 0:
            summation += (predictions[i] - value)**2
            non_zero += 1
    return summation/non_zero

In [18]:
print MSE(movie_avg_test, test_value)
print MSE(user_avg_test, test_value)
print MSE_non_zero(movie_avg_test, test_value)
print MSE_non_zero(user_avg_test, test_value)

0.0307532696129
0.0303709616256
13.796825834
13.4713849259


In [26]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(features_train, train_value)
predictions = lr.predict(features_test)

In [34]:
print lr.score(features_test, test_value)

0.0513487103814


In [27]:
print MSE(predictions, test_value)
print MSE_non_zero(predictions, test_value)

0.0297568698712
12.9444972069


In [30]:
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(features_train, train_value)
predictions = ridge.predict(features_test)

In [35]:
print ridge.score(features_test, test_value)

0.0513486075259


In [31]:
print MSE(predictions, test_value)
print MSE_non_zero(predictions, test_value)

0.0297568730932
12.9445485675


In [32]:
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(features_train, train_value)
predictions = lasso.predict(features_test)

In [33]:
print MSE(predictions, test_value)
print MSE_non_zero(predictions, test_value)

0.030566601058
13.9143903203


In [36]:
print lasso.score(features_test, test_value)

0.0255344179851
